In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import data_handling

In [26]:
import data_preprocessing

In [4]:
data = pd.read_csv("./test_dataset/loan_data.csv")

In [15]:
filename = "./test_dataset/loan_data.csv"

In [27]:
data_preprocessing.process_data(filename)

(array([[ 1.    ,  2.    ,  0.1189, ..., 52.1   ,  0.    ,  0.    ],
        [ 1.    ,  1.    ,  0.1071, ..., 76.7   ,  0.    ,  0.    ],
        [ 1.    ,  2.    ,  0.1357, ..., 25.6   ,  1.    ,  0.    ],
        ...,
        [ 0.    ,  2.    ,  0.1071, ..., 82.9   ,  8.    ,  0.    ],
        [ 0.    ,  4.    ,  0.16  , ...,  3.2   ,  5.    ,  0.    ],
        [ 0.    ,  2.    ,  0.1392, ..., 57.    ,  6.    ,  0.    ]]),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int64))

In [24]:
def process_data(filename):
    df = pd.read_csv(filename)

    substrings_to_remove = ['year', 'month', 'number', 'id', 'timestamp', 'index', 'text', 'period', 'counter']
    df.drop(columns=[col for col in df.columns if any(substring in col.lower() for substring in substrings_to_remove)], inplace=True, errors='ignore')

            # remove 'b' and ' ' in original dataset
            # df = df.applymap(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
    df = df.map(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)

            # missing values
    if df.isnull().values.any():
        print(f"{df} has missing values")
        for column in df.columns:
            if df[column].dtype == 'object':
                        # mode for categorical variables
                most_common = df[column].mode()[0]
                df[column].fillna(most_common, inplace=True)
            else:
                        # mean for numeric variables
                mean_value = df[column].mean()
                df[column].fillna(mean_value, inplace=True)
                df[column] = df[column].round(1)

            # If target column is not the last column, move to the last
    class_column = None
    for col in df.columns:
        if 'class' in col.lower() or col == 'Home/Away' in col:
            class_column = col
            break
    if class_column and df.columns[-1] != class_column:
        class_col = df[class_column]
        df = df.drop(columns=[class_column])
        df['cls'] = class_col
    else:
        df.rename(columns={df.columns[-1]: 'cls'}, inplace=True)
            

    # Assign numeric labels based on class frequency
    class_counts = df['cls'].value_counts(ascending=False)
    class_mapping = {cls: i for i, cls in enumerate(class_counts.index)}
    df['cls'] = df['cls'].map(class_mapping)

            
    label_encoders = {}
    for column in df.columns:
        if df[column].dtype == 'object':
            le = LabelEncoder()
            df[column] = le.fit_transform(df[column])
            label_encoders[column] = le
    
    target = df.iloc[:, -1].values  # Convert the last column to Numpy array
    data = df.iloc[:, :-1].values    # Convert all but the last column to Numpy array

    data = pd.DataFrame(data).apply(pd.to_numeric, errors='coerce').fillna(np.nan).values
    # target = pd.to_numeric(target, errors='coerce').fillna(np.nan).astype(int).values
    return data, target
    # return df
    

In [25]:
data, target = process_data(filename)
data, target

(array([[ 1.    ,  2.    ,  0.1189, ..., 52.1   ,  0.    ,  0.    ],
        [ 1.    ,  1.    ,  0.1071, ..., 76.7   ,  0.    ,  0.    ],
        [ 1.    ,  2.    ,  0.1357, ..., 25.6   ,  1.    ,  0.    ],
        ...,
        [ 0.    ,  2.    ,  0.1071, ..., 82.9   ,  8.    ,  0.    ],
        [ 0.    ,  4.    ,  0.16  , ...,  3.2   ,  5.    ,  0.    ],
        [ 0.    ,  2.    ,  0.1392, ..., 57.    ,  6.    ,  0.    ]]),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int64))

In [6]:
df = pd.DataFrame(data)

substrings_to_remove = ['year', 'month', 'number', 'id', 'timestamp', 'index', 'text', 'period', 'counter']
df.drop(columns=[col for col in df.columns if any(substring in col.lower() for substring in substrings_to_remove)], inplace=True, errors='ignore')

        # remove 'b' and ' ' in original dataset
        # df = df.applymap(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
df = df.map(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)

        # missing values
if df.isnull().values.any():
    print(f"{df} has missing values")
    for column in df.columns:
        if df[column].dtype == 'object':
                    # mode for categorical variables
            most_common = df[column].mode()[0]
            df[column].fillna(most_common, inplace=True)
        else:
                    # mean for numeric variables
            mean_value = df[column].mean()
            df[column].fillna(mean_value, inplace=True)
            df[column] = df[column].round(1)

        # If target column is not the last column, move to the last
class_column = None
for col in df.columns:
    if 'class' in col.lower() or col == 'Home/Away' in col:
        class_column = col
        break
if class_column and df.columns[-1] != class_column:
    class_col = df[class_column]
    df = df.drop(columns=[class_column])
    df['cls'] = class_col
else:
    df.rename(columns={df.columns[-1]: 'cls'}, inplace=True)
        

        # Assign numeric labels based on class frequency
class_counts = df['cls'].value_counts(ascending=False)
class_mapping = {cls: i for i, cls in enumerate(class_counts.index)}
df['cls'] = df['cls'].map(class_mapping)

        
label_encoders = {}
for column in df.columns:
    if df[column].dtype == 'object':
        le = LabelEncoder()
        df[column] = le.fit_transform(df[column])
        label_encoders[column] = le

df
# df.to_csv("./df.csv", index=False)

,credit.policy,purpose,int.rate,installment,log.annual.inc,dti,fico,days.with.cr.line,revol.bal,revol.util,inq.last.6mths,delinq.2yrs,cls
0,1,2,0.1189,829.10,11.350407,19.48,737,5639.958333,28854,52.1,0,0,0
1,1,1,0.1071,228.22,11.082143,14.29,707,2760.000000,33623,76.7,0,0,0
2,1,2,0.1357,366.86,10.373491,11.63,682,4710.000000,3511,25.6,1,0,0
3,1,2,0.1008,162.34,11.350407,8.10,712,2699.958333,33667,73.2,1,0,0
4,1,1,0.1426,102.92,11.299732,14.97,667,4066.000000,4740,39.5,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9573,0,0,0.1461,344.76,12.180755,10.39,672,10474.000000,215372,82.1,2,0,0
9574,0,0,0.1253,257.70,11.141862,0.21,722,4380.000000,184,1.1,5,0,0
9575,0,2,0.1071,97.81,10.596635,13.09,687,3450.041667,10036,82.9,8,0,0
9576,0,4,0.1600,351.58,10.819778,19.18,692,1800.000000,0,3.2,5,0,0


In [7]:
df.to_csv("./df.csv", index=False)